In [94]:
import functools
import operator
import itertools
import math
import sys
import json
import re
from enum import Enum
from collections import deque
from dataclasses import dataclass
from typing import Dict,List
import pulp

data = open("sample_input.txt").read().splitlines()


operations = {
    "+": operator.add,
    "-": operator.sub,
    "/": operator.truediv,
    "*": operator.mul,
}

class Monkey():
    waiting_on: List["Monkey"]
    value: int|bool
    l: str|None
    r: str|None
    left: "Monkey"
    right: "Monkey"
    op:any
    op_name: str
    paths: List[str]

    def __init__(self, line:str):
        self.paths = []
        self.line = line
        parts = line.split()
        self.name = parts[0].strip(':')
        if len(parts) == 2:
            self.value = int(parts[1])
        else:
            self.value = False
            self.l, self.r = parts[1], parts[3]
            self.op_name = parts[2]
            self.op = operations[self.op_name]


monkeys = [Monkey(line) for line in data]
monkey_map = {m.name:m for m in monkeys}

for m in monkeys:
    if m.value:
        continue
    m.left = monkey_map[m.l]
    m.right = monkey_map[m.r]

def show_monkey(m, depth = 0, max_depth = 10000000000) -> str:
    if isinstance(m, int):
        return f"{m}"
    if m.value :
        return f"{' '*depth}{m.name}: {m.value}"
    if depth >= max_depth:
        return f"{' '*depth}{m.name} ------- MAX DEPTH"

    l = show_monkey(m.left, depth+1, max_depth)
    r = show_monkey(m.right, depth+1, max_depth)
    return f"{' '*depth}{m.name}: {m.op_name}\n{l}\n{r}"



In [101]:
def get_value(monkey: Monkey) -> int:
    if monkey.value:
        return monkey.value

    left_value = get_value(monkey.left)
    right_value = get_value(monkey.right)
    return monkey.op(left_value, right_value)

root = monkey_map["root"]
#print(show_monkey(root))
result = get_value(root)
print(f"part 1: {result}")

part 1: 84244467642604.0


In [106]:
from sympy import symbols, Eq, simplify

def get_value_or_monkey(monkey: Monkey) -> int | Monkey:
    
    if monkey.name == "humn":
        return monkey
    
    if monkey.value:
        return monkey.value    

    left = get_value_or_monkey(monkey.left)
    right = get_value_or_monkey(monkey.right)

    if isinstance(left, int) and isinstance(right,int):
        return monkey.op(left,right)

    return monkey

def assign_value(monkey: Monkey, x: int) -> int:
    if monkey.name == "humn":
        monkey.value = x
        return x

    if monkey.value:
        return
    
    y = get_value_or_monkey(monkey.left)
    z = get_value_or_monkey(monkey.right)

    if isinstance(y, int) and not isinstance(z, int):
        if monkey.op_name == "+":
            z = x - y
        elif monkey.op_name == "-":
            z = y - x
        elif monkey.op_name == "*":
            z = x / y
        else:
            z = y / x
        z = assign_value(monkey.right, z)
    elif isinstance(z,int) and not isinstance(y,int):
        if monkey.op_name == "+":
            y = x - z
        elif monkey.op_name == "-":
            y = x + z
        elif monkey.op_name == "*":
            y = x / z
        else:
            y = x * z
        y = assign_value(monkey.left, y)
    else:
        print(f"multiple unassigned monkeys at {monkey.name}")
        return 0

    monkey.value = monkey.op(y,z)
    return monkey.value

x = symbols("x")

def make_expr(monkey: Monkey):
    if monkey.name == "humn":
        return x

    if monkey.value:
        return monkey.value
    
    y = make_expr(monkey.left)
    z = make_expr(monkey.right)


    if monkey.op_name == "+":
        return y + z
    elif monkey.op_name == "-":
        return y - z
    elif monkey.op_name == "*":
        return y * z
    else:
        return y / z

root = monkey_map["root"]

humn = monkey_map["humn"]

left_expr = make_expr(root.left)
right_expr = make_expr(root.right)

print(left_expr)
print(right_expr)

print(Eq(left_expr, right_expr))

print(simplify(Eq(left_expr, right_expr)))

#print(show_monkey(root, max_depth=2))



68634163976949.2 - 14.1037037037037*x
15610303684582.0
Eq(68634163976949.2 - 14.1037037037037*x, 15610303684582.0)
Eq(x, 3759569926192.0)
